In [35]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

In [36]:
date_cols = [
    'Auftragseingang', 
    'Auftragsende_SOLL',
    'AFO_Start_SOLL',
    'AFO_Ende_SOLL',
    'AFO_Start_IST',
    'AFO_Ende_IST',
    'Auftragsende_IST' 
]

date_cols_eval = [
    'Auftragseingang', 
    'Auftragsende_SOLL',
    'AFO_Start_SOLL',
    'AFO_Ende_SOLL',
    'AFO_Start_IST',
]

data_train = pd.read_csv(
    '../data/processed/data_cleaned_maschinen_2.csv',
    parse_dates=date_cols,
    sep=',',
    low_memory=False
)

data_public = pd.read_csv(
    '../data/raw/df_eval_public_2025-11-03.csv',
    parse_dates=date_cols_eval,
    sep=',',
    low_memory=False
)

data_private = pd.read_csv(
    '../data/raw/df_eval_private_2025-11-03.csv',
    parse_dates=date_cols_eval,
    sep=',',
    low_memory=False
)

In [37]:
# 1) Info-AFO zählen
info_count = (
    data_train[data_train['Arbeitsschrittbezeichnung'] == 'Info']
    .groupby('AuftragsID')
    .size()
)

valid_info_ids = set(info_count[info_count == 1].index)
invalid_missing_info_ids = set(info_count[info_count == 0].index)
invalid_multiple_info_ids = set(info_count[info_count > 1].index)

print("=== INFO-AFO CHECK ===")
print("Total unique orders:", data_train['AuftragsID'].nunique())
print("Orders with exactly 1 Info-AFO:", len(valid_info_ids))
print("Orders with NO Info-AFO:", len(invalid_missing_info_ids))
print("Orders with MULTIPLE Info-AFO:", len(invalid_multiple_info_ids))
print()

# 2) Abgeschlossene Aufträge prüfen
finished_ids = set(
    data_train[~data_train['Auftragsende_IST'].isna()]['AuftragsID'].unique()
)
open_ids = set(
    data_train[data_train['Auftragsende_IST'].isna()]['AuftragsID'].unique()
)

print("=== AUFTRAGSENDE CHECK ===")
print("Finished orders:", len(finished_ids))
print("Open orders (no Auftragsende_IST):", len(open_ids))
print()

# 3) Schnittmenge bilden
train_ids = valid_info_ids & finished_ids

print("=== FINAL TRAINING IDS ===")
print("Final #training orders:", len(train_ids))
print("Dropped due to Info-AFO or missing IST:", 
      data_train['AuftragsID'].nunique() - len(train_ids))
print()

# 4) Optional: Beispiele anzeigen
print("Example dropped (missing Info-AFO):")
print(list(invalid_missing_info_ids)[:10])

print("\nExample dropped (multiple Info-AFO):")
print(list(invalid_multiple_info_ids)[:10])

print("\nExample dropped (open orders):")
print(list(open_ids - finished_ids)[:10])

# 5) Und hier dein finaler Trainingsdatensatz
train_df = data_train[data_train['AuftragsID'].isin(train_ids)].copy()


=== INFO-AFO CHECK ===
Total unique orders: 150368
Orders with exactly 1 Info-AFO: 146598
Orders with NO Info-AFO: 0
Orders with MULTIPLE Info-AFO: 0

=== AUFTRAGSENDE CHECK ===
Finished orders: 141822
Open orders (no Auftragsende_IST): 8546

=== FINAL TRAINING IDS ===
Final #training orders: 138068
Dropped due to Info-AFO or missing IST: 12300

Example dropped (missing Info-AFO):
[]

Example dropped (multiple Info-AFO):
[]

Example dropped (open orders):
[np.int64(131085), np.int64(131110), np.int64(131138), np.int64(131140), np.int64(131170), np.int64(131180), np.int64(131185), np.int64(131240), np.int64(131341), np.int64(131359)]


🟩 Kurzfassung: Was hast du entfernt?
Grund	Anzahl	Erklärung
❌ Keine Info-AFO	0	Perfekte Datenqualität
❌ Mehrfach-Info-AFO	0	Perfekte Datenqualität
❌ Offene Aufträge	8.546	Kein Auftragsende → kein Training möglich
❌ Aufträge ohne End-IST (aber nicht offiziell offen)	3.754	Label fehlt → kein Training möglich
➡️ Gesamt entfernt	12.300	Darf nicht ins Training
138.068 Aufträge bleiben übrig → supervised trainierbar.

In [38]:
train_df.to_csv('../data/processed/data_cleaned_3.csv', index=False)

In [39]:
data_public.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4273 entries, 0 to 4272
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   AuftragsID                 4273 non-null   int64         
 1   BauteilID                  4273 non-null   int64         
 2   Bauteilbezeichnung         4273 non-null   object        
 3   Auftragseingang            4273 non-null   datetime64[ns]
 4   Priorität                  4273 non-null   int64         
 5   Auftragsende_SOLL          4273 non-null   datetime64[ns]
 6   Arbeitsschritt             4273 non-null   int64         
 7   Arbeitsschrittbezeichnung  4273 non-null   object        
 8   AFO_Start_SOLL             4273 non-null   datetime64[ns]
 9   AFO_Ende_SOLL              4273 non-null   datetime64[ns]
 10  AFO_Start_IST              35 non-null     datetime64[ns]
 11  MaschinenID                4020 non-null   object        
 12  Maschi

In [40]:
data_private.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4273 entries, 0 to 4272
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   AuftragsID                 4273 non-null   int64         
 1   BauteilID                  4273 non-null   int64         
 2   Bauteilbezeichnung         4273 non-null   object        
 3   Auftragseingang            4273 non-null   datetime64[ns]
 4   Priorität                  4273 non-null   int64         
 5   Auftragsende_SOLL          4273 non-null   datetime64[ns]
 6   Arbeitsschritt             4273 non-null   int64         
 7   Arbeitsschrittbezeichnung  4273 non-null   object        
 8   AFO_Start_SOLL             4273 non-null   datetime64[ns]
 9   AFO_Ende_SOLL              4273 non-null   datetime64[ns]
 10  AFO_Start_IST              26 non-null     datetime64[ns]
 11  MaschinenID                4021 non-null   object        
 12  Maschi

In [41]:
def clean_data(df):
    df.drop()